In [1]:
import sys
import pandas as pd
import numpy as np
import os
from os.path import join
from matplotlib import pyplot as plt
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
from collections import OrderedDict
from collections import defaultdict
import ast
import gc
import Bio.PDB
from Bio import SeqIO
from IPython.display import display
from collections import Counter 
import itertools
from IPython.html.widgets.widget_float import FloatProgress

%matplotlib inline
notebook_prefix = 'ks01'

/Users/karen/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/Users/karen/anaconda/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


## Orienting

In [2]:
def find_folder_upstream(folder_name, max_iterations=50):
    current_folder = os.getcwd()
    counter = 0
    while os.path.basename(current_folder) != folder_name and counter < max_iterations:
        current_folder = os.path.dirname(current_folder)
        counter += 1
    if not counter < max_iterations:
        return None
    return current_folder

In [3]:
root_folder_name = 'HIS3InterspeciesEpistasis'
root_folder = find_folder_upstream(root_folder_name)
if not root_folder:
    print 'Did not find root folder for our github repository.\nPlease run "ks01" notebook from a script inside the HIS3InterspeciesEpistasis folder!'

data_folder = os.path.join(root_folder, 'Data', '')
small_tables_folder = os.path.join(root_folder, 'Data_Small_Tables', '')
analysis_folder = os.path.join(root_folder, 'Analysis', '')
karen_folder = os.path.join(analysis_folder, 'Karen', '')
figures_folder = os.path.join(karen_folder, 'figures', '')
files_dump_folder = os.path.join(karen_folder, 'files_dump', '')
structure_predictions_folder = os.path.join(files_dump_folder, 'structure_predictions', '')
pymol_sessions_folder = os.path.join(files_dump_folder, 'structure_visualizations/pymol_sessions/', '')

## General stuff

In [4]:
old_dir = os.getcwd()
os.chdir(karen_folder)
%run 'functions_dump.py'
os.chdir(old_dir)
image_counter = Counter()

In [5]:
segment_colors_list = [('S1','#00aba9'), ('S2','#ff0097'), ('S3','#a200ff'), ('S4','#b8d000'), ('S5','#1ba1e2'), ('S6','#f09609'), 
('S7','#edc951'), ('S8','#cc2a36'), ('S9','#4f372d'), ('S10','#0c457d'), ('S11','#616161'), ('S12','#800080')]
segment_colors = OrderedDict(segment_colors_list)

### Working with positions

In [6]:
def remove_gaps(seq):
    return ''.join([c for c in seq if c != '-'])

def get_wt_position(position_in_alignment):
    truncated_seq = remove_gaps(aligned_Scer[:position_in_alignment+1])
    return len(truncated_seq)

def relative_to_absolute_numbering(mutation, segment, df_with_data):
    display(df_with_data[segment][df_with_data[segment].mut_list == mutation][['mut_list', 'mut_list_Scer']])
    return df_with_data[segment][df_with_data[segment].mut_list == mutation].mut_list_Scer.values[0]

In [7]:
Scer_Uniprot = open(os.path.join(files_dump_folder, 'HIS3_saccharomyces_cerevisiae_from_Uniprot_P06633.txt')).read().rstrip()
alignment_file = os.path.join(small_tables_folder, 'aa_seq.txt')
for seq_record in SeqIO.parse(alignment_file, 'fasta'):
    if seq_record.id == 'Scer':
        aligned_Scer = str(seq_record.seq)
        break
assert remove_gaps(aligned_Scer) == Scer_Uniprot

In [8]:
positions = pd.read_table(os.path.join(small_tables_folder, 'positions.csv'))
positions = positions.set_index('Unnamed: 0').transpose().reset_index()
positions.rename(columns={'index' : 'segment', 'positions':'positions_alignment'}, inplace=True)
positions.reset_index(drop=True, inplace=True)
positions['positions_alignment'] = positions['positions_alignment'].apply(lambda s: ast.literal_eval(s))
positions['positions_Uniprot_P06633'] = positions.positions_alignment.apply(lambda l: [get_wt_position(p) for p in l])
segment_names = positions.segment.values # not explicitly sorted
positions.to_csv(os.path.join(files_dump_folder, 'information_about_segments.csv'), index=False)
positions[:3]

Unnamed: 0,segment,wt1,positions_alignment,len1,len2,start1,end1,wt2,start2,end2,start_Scer,end_Scer,positions_Uniprot_P06633
0,S1,EALGAVRGVK,"[140, 141, 142, 143, 144, 145, 146, 147, 148, ...",10,10,140,149,EALSRAVVDL,160,169,106,135,"[106, 107, 108, 109, 110, 111, 112, 113, 114, ..."
1,S2,SNRPYAVVE,"[170, 171, 172, 173, 174, 175, 176, 177, 178, ...",9,9,170,178,LSCEMIPHF,189,197,136,163,"[136, 137, 138, 139, 140, 141, 142, 143, 144, ..."
2,S3,LGLQREKVGD,"[179, 180, 181, 182, 183, 184, 185, 186, 187, ...",10,7,179,188,LESFAEA,198,204,145,170,"[145, 146, 147, 148, 149, 150, 151, 152, 153, ..."


In [9]:
title = 'Positions of segments according to Uniprot sequence'
plot_better(grid='', height=2)
old_y = 2
for row in positions.iterrows():
    for position in row[1].positions_Uniprot_P06633:
        new_y = np.random.choice([1,2])
        while new_y == old_y:
            new_y = np.random.choice([1,2])
    x = row[1].positions_Uniprot_P06633
    plt.plot(x, [new_y for e in x], '.', lw=3, alpha=0.7, label=row[1].segment, color=segment_colors[row[1].segment])
    plt.text(np.median(x), new_y + 0.3, row[1].segment)
    old_y = new_y
plt.ylim(0,4)
plt.yticks([])
plt.xlabel('Positions according to Uniprot sequence')
plt.title(title)
plt.close()

### Adding from_aa to notation of mutations

In [11]:
def get_full_mutation(mutation):
    if mutation == '' or not mutation > 0:
        return ''
    position = int(mutation[:-1])
    assert Scer_Uniprot[position-1] != mutation[-1]
    return Scer_Uniprot[position-1] + mutation

def convert_to_full_mutations(mut_combination):
    if mut_combination == '' or not mut_combination > 0:
        return ''
    return ':'.join(get_full_mutation(mutation) for mutation in mut_combination.split(':'))

## Amino acid properties table

In [ ]:
# amino acid codes
aa_long = ['alanine', 'cysteine', 'aspartic acid', 'glutamic acid', 'phenylalanine', 'glycine', 
           'histidine', 'isoleucine', 'lysine', 'leucine', 'methionine', 'asparagine', 'proline',
           'glutamine', 'arginine', 'serine', 'threonine', 'valine', 'tryptophan', 'tyrosine', 'unknown_aa'] 
aa1 = list("ACDEFGHIKLMNPQRSTVWYX")
aa3 = "ALA CYS ASP GLU PHE GLY HIS ILE LYS LEU MET ASN PRO GLN ARG SER THR VAL TRP TYR XXX".split()
aa123 = dict(zip(aa1, aa3))
aa321 = dict(zip(aa3, aa1))
aa_full_to_1 = dict(zip(aa_long, aa1))
aa_1_to_full = dict(zip(aa1, aa_long))

In [ ]:
# Original table http://www.proteinsandproteomics.org/content/free/tables_1/table08.pdf

aa_properties = pd.read_table(os.path.join(files_dump_folder, 'properties_of_amino_acids.csv'), sep=';')

aa_properties['Fraction of buried among this aa'] = aa_properties['Percent buried residues'].apply(lambda s: float(s.split()[0]))
aa_properties['Fraction of buried among all buried'] = aa_properties['Percent buried residues'].apply(lambda s: float(s.split()[1][1:-1]))

aa_properties['Polarity average ranking'] = aa_properties['Ranking of amino acid polarities'].apply(lambda s: float(s.split()[0]))
aa_properties['Polarity ranking (Radzicka and Wolfenden 1988)'] = aa_properties['Ranking of amino acid polarities'].apply(lambda s: float(s.split()[1][1:-1]))

aa_properties['amino_acid'] = aa_properties['Amino acid residue'].apply(lambda s: s.lower())
aa_properties.set_index('amino_acid', inplace=True)

aa_properties.drop(['Unnamed: 8', 'Unnamed: 10', 'Percent buried residues', 'Ranking of amino acid polarities', 'Amino acid residue'], inplace=True, axis=1)
aa_properties = aa_properties.astype(np.float, raise_on_error=False)

quantitative_properties = [p for p in aa_properties.columns if 'pKa' not in p]

def get_aa_properties_by_single_letter_code(aa_1):
    return aa_properties.loc[aa_1_to_full[aa_1]]